<a href="https://colab.research.google.com/github/ruslanmv/Open-Source-LLM-Chatbot/blob/master/Open_Source_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Colab Pro notebook from https://github.com/ruslanmv/Open-Source-LLM-Chatbot**

# Open Source LLM **Chatbot**


In [ ]:
#@markdown # Connect Google Drive
from google.colab import drive
from IPython.display import clear_output
import ipywidgets as widgets
import os

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
Shared_Drive = "" #@param {type:"string"}
#@markdown - Leave empty if you're not using a shared drive

print("[0;33mConnecting...")
drive.mount('/content/gdrive')

if Shared_Drive!="" and os.path.exists("/content/gdrive/Shareddrives"):
  mainpth="Shareddrives/"+Shared_Drive
else:
  mainpth="MyDrive"

clear_output()
inf('\u2714 Done','success', '50px')

#@markdown ---

In [8]:
#@markdown # Install/Update Open Source LLM Chatbot repo
from IPython.utils import capture
from IPython.display import clear_output
from subprocess import getoutput
import ipywidgets as widgets
import sys
import fileinput
import os
import time
import base64
import gdown
from gdown.download import get_url_from_gdrive_confirmation
import requests
from urllib.request import urlopen, Request
from urllib.parse import urlparse, parse_qs, unquote
from tqdm import tqdm
import six
if not os.path.exists("/content/gdrive"):
  print('[1;31mGdrive not connected, using temporary colab storage ...')
  time.sleep(4)
  mainpth="MyDrive"
  !mkdir -p /content/gdrive/$mainpth
  Shared_Drive=""

if Shared_Drive!="" and not os.path.exists("/content/gdrive/Shareddrives"):
  print('[1;31mShared drive not detected, using default MyDrive')
  mainpth="MyDrive"

with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  fgitclone = "git clone --depth 1"
  %mkdir -p /content/gdrive/$mainpth/llm
  %cd /content/gdrive/$mainpth/llm

  !git clone -q --branch master https://github.com/ruslanmv/Open-Source-LLM-Chatbot.git /content/gdrive/$mainpth/llm/chatbot/
  !git fetch
  !git pull
  !mkdir -p /content/gdrive/$mainpth/llm/chatbot/cache/
  !pip install -r /content/gdrive/$mainpth/llm/chatbot/requirements.txt
  os.environ['TRANSFORMERS_CACHE']=f"/content/gdrive/{mainpth}/llm/chatbot/cache"
  os.environ['TORCH_HOME'] = f"/content/gdrive/{mainpth}/llm/chatbot/cache"

with capture.capture_output() as cap:
  %cd /content/gdrive/$mainpth/llm/chatbot/
clear_output()
inf('\u2714 Done','success', '50px')

#@markdown ---

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

In [12]:
#@markdown # Model Download/Load
from huggingface_hub import hf_hub_download
Use_Temp_Storage = False #@param {type:"boolean"}
#@markdown - If not, make sure you have enough space on your gdrive

#@markdown ---

Model_Version = "Mistral-7B-Instruct-v0.2" #@param ["Mistral-7B-Instruct-v0.2", "CodeLlama-7B", "Llama-2-13B-chat", "Falcon-7B-Instruct","zephyr-7B-beta","vicuna-7B-v1.5"]

if Use_Temp_Storage:
   MODELS_PATH = "./models"
else:
    MODELS_PATH ='/content/gdrive/'+mainpth+'/llm/chatbot'+'/models/'

def llmdl(ver, Use_Temp_Storage):

  repo_id=''
  if(ver=="Llama-2-13B-chat"):
        repo_id="TheBloke/Llama-2-13B-chat-GGUF"
        filename="llama-2-13b-chat.Q4_K_M.gguf"
  elif(ver=="Mistral-7B-Instruct-v0.2") :
        repo_id="TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
        filename="mistral-7b-instruct-v0.2.Q4_K_M.gguf"
  elif(ver=="zephyr-7B-beta"):
        repo_id="TheBloke/zephyr-7B-beta-GGUF "
        filename="zephyr-7b-beta.Q4_K_M.gguf"
  elif(ver=="vicuna-7B-v1.5"):
        repo_id="TheBloke/vicuna-7B-v1.5-GGUF"
        filename="vicuna-7b-v1.5.Q4_K_M.gguf"
  elif(ver=="Falcon-7B-Instruct"):
        repo_id="TheBloke/Falcon-7B-Instruct-GGML"
        filename="falcon-7b-instruct.ggccv1.q4_1.bin"
  elif(ver=="CodeLlama-7B"):
        repo_id="TheBloke/CodeLlama-7B-GGUF"
        filename="codellama-7b.Q4_K_M.gguf"
  if Use_Temp_Storage:
      os.makedirs('/content/models', exist_ok=True)
      model='/content/models/'+ver
  else:
      model=MODELS_PATH+ver
  link=repo_id
  if not os.path.exists(model):
    model_path = hf_hub_download(
    repo_id= repo_id,
    filename= filename,
    resume_download=True,
    cache_dir=MODELS_PATH,)

    if os.path.exists(model_path):
      clear_output()
      inf('\u2714 Done','success', '50px')
    else:
      inf('\u2718 Something went wrong, try again','danger', "250px")
  else:
      clear_output()
      inf('\u2714 Model already exists','primary', '300px')

  return model





inf('\u2718 Downloading files','success', "400px")
PATH_to_MODEL=llmdl(Model_Version, Use_Temp_Storage)

if os.path.exists(str(PATH_to_MODEL)):
  inf('\u2714 Using the custom model.','success', '200px')
  model=PATH_to_MODEL

#@markdown ---

Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

In [ ]:
#@markdown # Start Chatbot
from IPython.utils import capture
import time
import sys
import fileinput
from pyngrok import ngrok, conf
import re

Use_Cloudflare_Tunnel = False #@param {type:"boolean"}
#@markdown - Offers better gradio responsivity

Ngrok_token = "" #@param {type:"string"}

#@markdown - Input your ngrok token if you want to use ngrok server

User = "" #@param {type:"string"}
Password= "" #@param {type:"string"}
#@markdown - Add credentials to your Gradio interface (optional)

auth=f"--gradio-auth {User}:{Password}"
if User =="" or Password=="":
  auth=""


#with capture.capture_output() as cap:
#  %cd /content/gdrive/$mainpth/llm/chatbot/modules/

share=''
if Ngrok_token!="":
  ngrok.kill()
  srv=ngrok.connect(7860, pyngrok_config=conf.PyngrokConfig(auth_token=Ngrok_token) , bind_tls=True).public_url

  for line in fileinput.input('/usr/local/lib/python3.10/dist-packages/gradio/blocks.py', inplace=True):
    if line.strip().startswith('self.server_name ='):
        line = f'            self.server_name = "{srv[8:]}"\n'
    if line.strip().startswith('self.protocol = "https"'):
        line = '            self.protocol = "https"\n'
    if line.strip().startswith('if self.local_url.startswith("https") or self.is_colab'):
        line = ''
    if line.strip().startswith('else "http"'):
        line = ''
    sys.stdout.write(line)

elif Use_Cloudflare_Tunnel:
  with capture.capture_output() as cap:
    !pkill cloudflared
    time.sleep(4)
    !nohup cloudflared tunnel --url http://localhost:7860 > /content/srv.txt 2>&1 &
    time.sleep(4)
    with open('/content/srv.txt', "r") as file: text = file.read()
    srv= re.findall(r"https?://(?:\S+?\.)?trycloudflare\.com\S*", text)[0]

    for line in fileinput.input('/usr/local/lib/python3.10/dist-packages/gradio/blocks.py', inplace=True):
      if line.strip().startswith('self.server_name ='):
          line = f'            self.server_name = "{srv[8:]}"\n'
      if line.strip().startswith('self.protocol = "https"'):
          line = '            self.protocol = "https"\n'
      if line.strip().startswith('if self.local_url.startswith("https") or self.is_colab'):
          line = ''
      if line.strip().startswith('else "http"'):
          line = ''
      sys.stdout.write(line)

    !rm /content/srv.txt

else:
  share='--share'

try:
  if os.path.isfile(model):
    !python /content/gdrive/$mainpth/llm/chatbot/app.py
  else:
    !python /content/gdrive/$mainpth/llm/chatbot/app.py
except:
   !python /content/gdrive/$mainpth/llm/chatbot/app.py

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ./models/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:      